## Search Engine With Tools And Agents

Link: https://python.langchain.com/v0.1/docs/modules/tools/

Build in tools: https://python.langchain.com/v0.1/docs/integrations/tools/

Build an Agent with AgentExecutor (Legacy): https://python.langchain.com/docs/how_to/agent_executor/

Agent: https://python.langchain.com/v0.1/docs/modules/agents/

Quick Start: https://python.langchain.com/v0.1/docs/modules/agents/quick_start/

In [3]:
## Arxiv--Research
## Tools creation
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper

### Tool 1  - Wikipedia

In [6]:
## Used the inbuilt tool of wikipedia
api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

'wikipedia'

### Tool 2 - arxiv

In [8]:
api_wrapper_arxiv=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
print(arxiv.name)

arxiv


### Tool 3 - Custom tool - RAG Tool

In [9]:
## Custom tools[RAG Tool]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
# from langchain_openai import OpenAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [8]:
# loader=WebBaseLoader("https://docs.smith.langchain.com/")
# docs=loader.load()
# documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
# vectordb=FAISS.from_documents(documents,OpenAIEmbeddings())
# retriever=vectordb.as_retriever()
# retriever

In [10]:
loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectordb=FAISS.from_documents(documents,embeddings)
retriever=vectordb.as_retriever()
retriever

d:\Data Science\02-AgenticAIworkspace\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000022A1B2D7D10>, search_kwargs={})

In [11]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith-search","Search any information about Langsmith ")

retriever_tool.name

'langsmith-search'

### Combainaing all tools

In [12]:
tools=[wiki,arxiv,retriever_tool]

In [13]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\Data Science\\02-AgenticAIworkspace\\venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith-search', description='Search any information about Langsmith ', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000022A713B59E0>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000

## Run all this tools with Agents and LLM Models

Quick start: https://python.langchain.com/v0.1/docs/modules/agents/quick_start/

In [10]:
## Run all this tools with Agents and LLM Models

## Tools, LLM-->AgentExecutor
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import openai
load_dotenv()
import os

groq_api_key=os.getenv("GROQ_API_KEY")
openai.api_key=os.getenv("OPENAI_API_KEY")

llm=ChatGroq(groq_api_key=groq_api_key,model_name="Llama3-8b-8192")

### OpenAI tools

In [11]:
## Prompt Template
from langchain import hub
prompt=hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

Link: https://python.langchain.com/v0.1/docs/modules/agents/agent_types/openai_tools/#create-agent

#### Create Agent

In [12]:
## Agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

#### Run Agent

In [13]:
## Agent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [14]:
agent_executor.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith-search` with `{'query': 'Langsmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

LangSmith + LangChain OSSLangSmith integrates seamlessly with LangChain's open source frameworks langchain and langgraph, with no extra instrumentation needed.If you're already using either of these, see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with LangGraph.
Observability​
Observability is important for any software application, but especially so for LLM applications. LLMs are non-deterministic by nature, meaning they can produce unexpected results. This makes them trickier than normal to debug.
This is where LangSmith can help! LangSmith has LLM-native observability, allowing you to get meaningful insights from your application. LangSmith’s observability features have you covered throughout all stages of application development - from prototyping, to beta testing, to production.

Skip to main conten

{'input': 'Tell me about Langsmith',
 'output': 'Based on the output from the Langsmith-search tool, it appears that Langsmith is a platform for building production-grade Large Language Model (LLM) applications. Langsmith provides features such as observability, evaluation, and prompt engineering to help developers closely monitor and evaluate their applications, iterate on prompts, and ship quickly and with confidence.\n\nSome of the key features of Langsmith include:\n\n* Observability: Langsmith provides LLM-native observability, allowing developers to get meaningful insights from their application.\n* Evaluation: Langsmith allows developers to evaluate their application over production traffic, score application performance, and get human feedback on their data.\n* Prompt Engineering: Langsmith provides a set of tools designed to enable and facilitate prompt engineering, helping developers find the perfect prompt for their application.\n\nOverall, Langsmith seems to be a powerful p

In [19]:
agent_executor.invoke({"input":"What is machine learning"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data and thus perform tasks with
Invoking: `arxiv` with `{'query': 'machine learning'}`


Published: 2019-09-08
Title: Lecture Notes: Optimization for Machine Learning
Authors: Elad Hazan
Summary: Lecture notes on optimization for machine learning, derived from a course at
Princeton University and tutorials given in MLSS, Buenos Aires, as
Invoking: `langsmith-search` with `{'query': 'Elad Hazan'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

"revision_id": "beta"    },)import { Client, Run, Example } from "langsmith";import { evaluate } from "langsmith/evaluation";import { EvaluationResult } from "langsmith/evaluation";const client = new Client();// Define dataset: thes

{'input': 'What is machine learning',
 'output': "Thank you for the information about LangSmith! Based on this, I can provide a text response.\n\nMachine learning is a field of study in artificial intelligence that involves the development and use of algorithms that can learn from data and improve their performance on a task over time. These algorithms can be used to make predictions, classify objects, and make decisions, and they are widely used in many fields, including computer science, engineering, and healthcare.\n\nIt's great to see LangSmith as a tool that can assist with machine learning tasks, providing a platform for users to easily evaluate and improve their models. With LangSmith, users can create datasets, define evaluators, and run evaluations to test their models and improve their performance.\n\nI hope this response provides a good overview of machine learning and LangSmith!"}

In [20]:
agent_executor.invoke({"input":"What's the paper 1706.03762 about?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on c
Invoking: `wikipedia` with `{'query': 'Attention Is All You Need'}`


Page: Attention Is All You Need
Summary: "Attention Is All You Need" is a 2017 landmark research paper authored by eight scientists working at Google, that introduced a new deep learning architecture known as the transformer based on attention mechanThe Transformer is a neural network architecture that was first introduced in this paper. It's primarily used for natural language processing tasks, such as machine translation, text summarization, and language modeling. The Transformer architecture is based on self-attention mechanisms, which allow it to focus on specific parts of the

{'input': "What's the paper 1706.03762 about?",
 'output': "The Transformer is a neural network architecture that was first introduced in this paper. It's primarily used for natural language processing tasks, such as machine translation, text summarization, and language modeling. The Transformer architecture is based on self-attention mechanisms, which allow it to focus on specific parts of the input data while processing it. This architecture has been widely adopted in many NLP tasks and has achieved state-of-the-art results in many applications."}

### HW: ReAct Agent

Link: https://python.langchain.com/v0.1/docs/modules/agents/agent_types/react/

In [2]:
## Tools, LLM-->AgentExecutor
from langchain_groq import ChatGroq
from langchain.agents import AgentExecutor, create_react_agent
from langchain import hub
from dotenv import load_dotenv
load_dotenv()
import os

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

In [ ]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\Data Science\\02-AgenticAIworkspace\\venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith-search', description='Search any information about Langsmith ', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000002064E7F7600>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000

#### Create Agent


In [10]:
# Create llm
llm=ChatGroq(model="Llama3-8b-8192")

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react")

# Construct the ReAct agent
agent = create_react_agent(llm, tools, prompt)

agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={'tools': 'wikipedia - A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\narxiv - A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Quantitative Biology, Quantitative Finance, Statistics, Electrical Engineering, and Economics from scientific articles on arxiv.org. Input should be a search query.\nlangsmith-search(query: \'str\', *, retriever: \'BaseRetriever\' = VectorStoreRetriever(tags=[\'FAISS\', \'HuggingFaceEmbeddings\'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000206000A1610>, search_kwargs={}), document_prompt: \'BasePromptTemplate\

#### Run Agent

In [13]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={'tools': 'wikipedia - A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\narxiv - A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Quantitative Biology, Quantitative Finance, Statistics, Electrical Engineering, and Economics from scientific articles on arxiv.org. Input should be a search query.\nlangsmith-search(query: \'str\', *, retriever: \'BaseRetriever\' = VectorStoreRetriever(tags=[\'FAISS\', \'HuggingFaceEmbeddings\'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000206000A1610>, 

In [14]:
agent_executor.invoke({"input": "what is LangChain?"})



> Entering new AgentExecutor chain...
Thought: LangChain is a language chain model, and I'm not familiar with it, so I should search for more information.

Action: langsmith-search
Action Input: "LangChain"LangSmith + LangChain OSSLangSmith integrates seamlessly with LangChain's open source frameworks langchain and langgraph, with no extra instrumentation needed.If you're already using either of these, see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with LangGraph.
Observability​
Observability is important for any software application, but especially so for LLM applications. LLMs are non-deterministic by nature, meaning they can produce unexpected results. This makes them trickier than normal to debug.
This is where LangSmith can help! LangSmith has LLM-native observability, allowing you to get meaningful insights from your application. LangSmith’s observability features have you covered throughout all stages of application development - from prot

{'input': 'what is LangChain?',
 'output': 'LangChain is a company that develops LangSmith, a platform for building production-grade Large Language Models (LLMs).'}

In [15]:
agent_executor.invoke({"input":"What is machine learning"})



> Entering new AgentExecutor chain...
Thought: I need to answer a question about a general topic, so I'll use Wikipedia.

Action: wikipedia
Action Input: "machine learning"Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and genThought: This Wikipedia page seems to provide a good overview of machine learning.

Action: None
Invalid Format: Missing 'Action Input:' after 'Action:'It looks like I made a mistake! I forgot to include the Action Input after the Action.

Let me correct that:

Question: What is machine learning
Thought: I need to answer a question about a general topic, so I'll use Wikipedia.

Action: wikipedia
Action Input: "machine learning"Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data an

{'input': 'What is machine learning',
 'output': 'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generate predictions or decisions.\n\nNote: Since the question is about a general topic, I used Wikipedia to answer it.'}

In [16]:
agent_executor.invoke({"input":"What's the paper 1706.03762 about in short?"})



> Entering new AgentExecutor chain...
Thought: This question seems to be about a scientific paper, possibly in the field of Physics, Mathematics, or Computer Science.

Action: arxiv

Action Input: 1706.03762
Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on cHere's the response:

Question: What's the paper 1706.03762 about in short?

Thought: This question seems to be about a scientific paper, possibly in the field of Physics, Mathematics, or Computer Science.

Action: arxiv

Action Input: 1706.03762
Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on cHere's my response:

Question: What's the p

{'input': "What's the paper 1706.03762 about in short?",
 'output': 'The paper 1706.03762, titled "Attention Is All You Need", is about a new approach to sequence transduction models that replaces traditional recurrent and convolutional neural networks with self-attention mechanisms.'}